In [9]:
#IMPORT REQUIRED LIBRARIES

from sqlalchemy import create_engine

import re
import pandas as pd
import numpy as np
from plotnine import *
from pandas import DataFrame, read_csv
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

import snscrape.modules.twitter as sntwitter
import numpy as np
import seaborn as sns

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

import string
import re
import textblob
from textblob import TextBlob

from wordcloud import WordCloud, STOPWORDS
from emot.emo_unicode import UNICODE_EMOJI

lemmatizer = WordNetLemmatizer()
from wordcloud import ImageColorGenerator
from PIL import Image

import warnings
%matplotlib inline

from sqlalchemy import create_engine
import pymysql

ModuleNotFoundError: No module named 'wordcloud'

In [10]:
#MYSQL CONNECTIONS

mysql_user = "root"
mysql_password = "localhost"
mysql_database = "airline_passenger_system"
mysql_host = "127.0.0.1"
port = 3306
sqlEngine = create_engine(f'mysql+pymysql://{mysql_user}:{mysql_password}@{mysql_host}/{mysql_database}',pool_recycle=port)
dbConnection = sqlEngine.connect()

In [15]:
#REGEX FUNCTION DEFINED TO REMOVE JUNK CHARACTERS


def stripChar(x):
    try:
        return re.sub("[()\"]","",x)
    except:
        return np.nan


In [12]:
import os
print('Get current working directory : ', os.getcwd())
url = f"{os.getcwd()}/../../Data"

#MAIN_FILE INTO DATAFRAME

df_original = pd.read_csv(f"{url}/City/airport-codes.csv",header=0,encoding = 'unicode_escape')

#DROP EMPTY ROWS
df_original = df_original.dropna()

#DROP DUPLICATES
df_original = df_original.drop_duplicates(keep='first')

#REPLACE SPACE WITH UNDERSCORE FOR COLUMNS
df_original.columns = df_original.columns.str.replace(' ', '_')

#LOWERCASE ALL THE COLUMN NAMES
df_original.columns = [x.lower() for x in df_original.columns]

#RENAME COLUMN VALUES
df_original.rename(columns={'country_abbrev.':'country_code','airport_code':'iata_airport'}, inplace=True)

#Removing duplicate iata values
#df_original.drop_duplicates(subset = ['iata_airport'], keep = 'first', inplace = True) 
#Removing duplicate iata values
df_original.drop_duplicates(subset = ['country_code','country_name'], keep = 'first', inplace = True) 
df_original.drop_duplicates(subset = ['world_area_code'], keep = 'first', inplace = True)

df_original


Get current working directory :  /Users/skudli/Desktop/airline-passenger-satisfaction-system/Python Scripts/Data Ingestion Scripts


,city_name,iata_airport,airport_name,country_name,country_code,world_area_code
1,108 Mile Ranch,ZMH,108 Mile Ranch,Canada,CA,906
3,Aachen,AAH,Aachen/Merzbruck,Germany,DE,429
7,Aalborg,AAL,Aalborg,Denmark,DK,419
9,Aalesund,AES,Vigra,Norway,NO,465
15,Aasiaat,JEG,Aasiaat,Greenland,GL,975
...,...,...,...,...,...,...
16701,Singapore,SIN,Changi,Singapore,SG,776
17061,St Croix Island,STX,Alex Hamilton,Virgin Islands(U.S.),VI,4
17069,St Denis de la Reunion,RUN,Gillot,Reunion,RE,563
17109,St Lucia,UVF,Hewanorra,Saint Lucia,LC,276


In [13]:
#INPUT FILE FOR AIRPORT TABLE

df_airport = pd.read_csv(f"{url}/Airport/airport_data_02.txt",header = 0)


#df_airport.loc[df_airport['country_code'].isin('IN')]

# #DROP UNWANTED COLUMNS
# df_airport=df_airport.drop(['url','city','county','state','code'], axis=1)

#RENAMING COLUMN NAMES
df_airport.rename(columns={'region_name':'state','airport':'airport_name','iata':'iata_airport','icao':'icao_airport'}, inplace=True)

# df_airport=df_airport.drop(['url','city','county','state','code'], axis=1)

#Removing duplicate iata values
df_airport.drop_duplicates(subset = ['iata_airport'], keep = 'first', inplace = True) 
df_airport=df_airport.drop_duplicates(keep='first')

df_airport = df_airport[['iata_airport','icao_airport','airport_name','state','country_code','latitude','longitude']]

len(list(df_airport['country_code'].unique()))

df_airport

#WRITING THE TABLE TO DATABASE
# df_airport.to_sql(con=dbConnection,name='AIRPORT',if_exists='replace',index=False)

# print("Airport table added to database successfully")

,iata_airport,icao_airport,airport_name,state,country_code,latitude,longitude
0,SHJ,OMSJ,Sharjah International Airport,Ash Shariqah,AE,25.328600,55.517200
1,AZI,OMAD,Al Bateen Executive Airport,Abu Zaby,AE,24.428301,54.458099
2,FJR,OMFJ,Fujairah International Airport,Al Fujayrah,AE,25.112200,56.324001
3,XSB,OMBY,Sir Bani Yas Airport,Abu Zaby,AE,24.283611,52.580278
4,RKT,OMRK,Ras Al Khaimah International Airport,Ra's al Khaymah,AE,25.613501,55.938801
...,...,...,...,...,...,...,...
8978,GWE,FVTL,Thornhill Air Base,Midlands,ZW,-19.436399,29.861900
8979,BUQ,FVBU,Joshua Mqabuko Nkomo International Airport,Bulawayo,ZW,-20.017401,28.617901
8980,CHJ,FVCH,Chipinge Airport,Manicaland,ZW,-20.206699,32.628300
8981,KAB,FVKB,Kariba Airport,Mashonaland West,ZW,-16.519800,28.885000


In [43]:
#READING CITY FILE

df_city= pd.read_csv(f"{url}/City/airport-codes-2.csv",header=0)
df_city = df_city.dropna()
df_city=df_city.rename(columns={'Airport Code':'iata_airport'})
df_city

,City name,iata_airport,Airport Name,Country name,Country Abbrev.,World Area Code
1,108 Mile Ranch,ZMH,108 Mile Ranch,Canada,CA,906
3,Aachen,AAH,Aachen/Merzbruck,Germany,DE,429
5,Aachen,ZIU,Railway,Germany,DE,429
7,Aalborg,AAL,Aalborg,Denmark,DK,419
9,Aalesund,AES,Vigra,Norway,NO,465
...,...,...,...,...,...,...
20211,St. John's,YYT,St. John´s International Airport,Canada,CA,906
20213,New Orleans,MSY,Louis Armstrong New Orleans International Airport,United States,US,67
20215,Dammam,DMM,King Fahd International Airport,Saudi Arabia,SA,670
20217,Pittsburgh,PIT,Pittsburgh International Airport,United States,US,67


In [45]:
#CONSOLIDATED MERGE TABLE TO SERVE AS OUR MAIN DATAFRAME

#df_review_merge = merge_df.drop_duplicates(keep='first')
merge_df = pd.merge(df_airport, df_city, on='iata_airport', how='inner')
merge_df = merge_df.drop(['country_code','Airport Name'],axis=1)
merge_df = merge_df.rename(columns={'Country Abbrev.':'country_code','World Area Code':'world_area_code'})
merge_df = merge_df.drop_duplicates(keep='first')
merge_df['area_code_id'] = merge_df['country_code']+merge_df['world_area_code']
merge_df

#DROP EMPTY ROWS
merge_df = merge_df.dropna()

#REPLACE SPACE WITH UNDERSCORE FOR COLUMNS
merge_df.columns = merge_df.columns.str.replace(' ', '_')

#LOWERCASE ALL THE COLUMN NAMES
merge_df.columns = [x.lower() for x in merge_df.columns]

#REMOVE UNWANTED CHARACTERS FROM ROWS
merge_df.country_name = merge_df.country_name.apply(lambda x: stripChar(x))

merge_df

#len(list(merge_df['iata_airport'].unique()))

,iata_airport,icao_airport,airport_name,state,latitude,longitude,city_name,country_name,country_code,world_area_code,area_code_id
0,SHJ,OMSJ,Sharjah International Airport,Ash Shariqah,25.328600,55.517200,Sharjah,United Arab Emirates,AE,678,AE678
1,AZI,OMAD,Al Bateen Executive Airport,Abu Zaby,24.428301,54.458099,Abu Dhabi,United Arab Emirates,AE,678,AE678
2,FJR,OMFJ,Fujairah International Airport,Al Fujayrah,25.112200,56.324001,Al-Fujairah,United Arab Emirates,AE,678,AE678
3,RKT,OMRK,Ras Al Khaimah International Airport,Ra's al Khaymah,25.613501,55.938801,Ras Al Khaimah,United Arab Emirates,AE,678,AE678
4,DHF,OMAM,Al Dhafra Air Base,Abu Zaby,24.248199,54.547699,Abu Dhabi,United Arab Emirates,AE,678,AE678
...,...,...,...,...,...,...,...,...,...,...,...
8393,GWE,FVTL,Thornhill Air Base,Midlands,-19.436399,29.861900,Gweru,Zimbabwe,ZW,565,ZW565
8394,BUQ,FVBU,Joshua Mqabuko Nkomo International Airport,Bulawayo,-20.017401,28.617901,Bulawayo,Zimbabwe,ZW,565,ZW565
8395,CHJ,FVCH,Chipinge Airport,Manicaland,-20.206699,32.628300,Chipinge,Zimbabwe,ZW,565,ZW565
8396,KAB,FVKB,Kariba Airport,Mashonaland West,-16.519800,28.885000,Kariba,Zimbabwe,ZW,565,ZW565


In [38]:
#POPULATE COUNTRY TABLE 
#TABLE 1

df_country= pd.read_csv(f"{url}/Country code/country_code.txt",header=0)

df_country.rename(columns={'Name':'country_name','Code':'country_code'}, inplace=True)


#REPLACE SPACE WITH UNDERSCORE FOR COLUMNS
df_country.columns = df_country.columns.str.replace(' ', '_')

#LOWERCASE ALL THE COLUMN NAMES
df_country.columns = [x.lower() for x in df_country.columns]


#REMOVE UNWANTED CHARACTERS FROM ROWS
df_country.country_name = df_country.country_name.apply(lambda x: stripChar(x))

#CAPITALIZE
df_country.country_name=df_country.country_name.apply(lambda x: x.title())

#REMOVE DUPLICATES
df_country = df_country.drop_duplicates(keep='first')

#Not empty
#df_country = df_country[df_country['country_code'].str.contains('[a-zA-Z]$')]
df_country = df_country.dropna()

df_country

#WRITING THE TABLE TO DATABASE
df_country.to_sql(con=dbConnection,name='country',if_exists='append',index=False)

display(df_country)
print("Country table added to database successfully")

,country_name,country_code
0,Afghanistan,AF
1,Ã…Land Islands,AX
2,Albania,AL
3,Algeria,DZ
4,American Samoa,AS
...,...,...
244,Wallis And Futuna,WF
245,Western Sahara,EH
246,Yemen,YE
247,Zambia,ZM


Country table added to database successfully


In [39]:
#POPULATE COUNTRY AREA 
#TABLE 2

df_country_area = merge_df[['country_code','world_area_code','area_code_id']]
df_country_area = df_country_area.drop_duplicates(keep='first')
#merge_df_ext = merge_df_ext.rename(columns={'Country Abbrev.':'country_code','World Area Code':'world_area_code'})
#df_country['area_code_id'] = df_country.reset_index()

df_country_area = df_country_area[df_country_area['country_code'].str.contains('[a-zA-Z]$')]

# merge_df_ext.reset_index(inplace=True)
# merge_df_ext.index = merge_df_ext.index + 1
# merge_df_ext['area_code_id'] = merge_df_ext.index
# merge_df_ext = merge_df_ext.drop(['index'],axis=1)
# merge_df_ext

#WRITING THE TABLE TO DATABASE
df = pd.merge(df_country,df_country_area,how="left",on=['country_code'])
df = df.dropna(how='any')


df_country_area_final = df[['area_code_id','country_code','world_area_code']]


#WRITING THE TABLE TO DATABASE
df_country_area_final.to_sql(con=dbConnection,name='country_area',if_exists='append',index=False)

display(df_country_area_final)
print("COUNTRY_AREA table added to database successfully")
len(list(df['area_code_id'].unique()))

KeyError: "None of [Index(['country_code', 'world_area_code', 'area_code_id'], dtype='object')] are in the [columns]"

In [19]:
##POPULATE CITY TABLE
#TABLE 3

df_city = merge_df[['iata_airport','city_name','country_code']]
df_city = df_city.drop_duplicates(keep='first')
df_city=df_city.rename(columns={'iata_airport':'city_code'})
df_city['city_code'] = df_city['city_code'].str.upper()

df_city = df_city.drop_duplicates(keep='first')
df_city

df_city = df_city[df_city['country_code'].str.contains('[a-zA-Z]$')]


#WRITING THE TABLE TO DATABASE

df_country_code = df_country[['country_code']]
df = pd.merge(df_country_code,df_city,how="left",on=['country_code'])
df.drop_duplicates(subset = ['city_code'], keep = 'first', inplace = True) 
df = df.dropna(how='any')

df = df[['city_code','city_name','country_code']]
df_city_final=df
display(df_city_final)


#WRITING THE TABLE TO DATABASE
df_city_final.to_sql(con=dbConnection,name='city',if_exists='append',index=False)

print("City table added to database successfully")
len(list(df_city_final['city_code'].unique()))

,city_code,city_name,country_code
0,FAH,Farah,AF
1,MMZ,Maimana,AF
2,KWH,Khwahan,AF
3,SBF,Sardeh Band,AF
4,KBL,Kabul,AF
...,...,...,...
7279,GWE,Gweru,ZW
7280,BUQ,Bulawayo,ZW
7281,CHJ,Chipinge,ZW
7282,KAB,Kariba,ZW


City table added to database successfully


7252

In [40]:
#POPULATE AIRPORT TABLE
#TABLE 4

df_airport = merge_df[['iata_airport','icao_airport','airport_name','state','latitude','longitude','area_code_id']]
df_airport = df_airport.drop_duplicates(keep='first')
df_airport
df_airport=df_airport.rename(columns={'iata_airport':'city_code'})

df_city_code = df_city_final[['city_code']]
df = pd.merge(df_city_code,df_airport,how="left",on=['city_code'])
df.drop_duplicates(subset = ['city_code'], keep = 'first', inplace = True) 
df = df.dropna(how='any')
df=df.rename(columns={'city_code':'iata_airport','icao_code':'icao_airport'})
df

# df = df[['city_code','city_name','country_code']]
# df_city_final=df
# display(df_city_final)

#WRITING THE TABLE TO DATABASE
df.to_sql(con=dbConnection,name='airport',if_exists='replace',index=False)

display(df)
print("airport table added to database successfully")

#len(list(df['iata_airport'].unique()))


KeyError: "None of [Index(['iata_airport', 'icao_airport', 'airport_name', 'state', 'latitude',\n       'longitude', 'area_code_id'],\n      dtype='object')] are in the [columns]"

In [21]:
#MERGING TABLES TO POPULATE AIRLINE TABLE
#TABLE 5

df_airlines_three = pd.read_csv(f"{url}/Airlines/airlines-data-02.csv",header=None, names=['id','name','alias','iata','icao','callsign','country','active'])
df_airlines_three = df_airlines_three.drop([0])
df_airlines_three = df_airlines_three.drop([1])
df_airlines_three = df_airlines_three.drop([2])
df_airlines_three = df_airlines_three.drop([3])

#REMOVING SPACES
df_airlines_three=df_airlines_three.rename(columns=lambda x: x.strip())

#LOWERCASE COLUMN NAMES
df_airlines_three.columns = [x.lower() for x in df_airlines_three.columns]

df_airlines_three=df_airlines_three.drop(['id','alias','callsign'],axis=1)


#RENAME COLUMNS
df_airlines_three.rename(columns={'name':'airline_name','iata':'iata_airline','icao':'icao_airline','country':'country_name'}, inplace=True)

df_airlines_three = df_airlines_three.dropna()

df_airlines_three = df_airlines_three[(df_airlines_three.iata_airline != "\\N")]
df_airlines_three = df_airlines_three[(df_airlines_three.country_name != "\\N")]
df_airlines_three = df_airlines_three[(df_airlines_three.icao_airline != "\\N")]

df_airlines_three = df_airlines_three[df_airlines_three['iata_airline'].str.contains('[a-zA-Z]$')]
df_airlines_three = df_airlines_three[df_airlines_three['icao_airline'].str.contains('[a-zA-Z]$')]

df_airlines_three = df_airlines_three.drop_duplicates(keep='first')
#Removing duplicate iata values
df_airlines_three.drop_duplicates(subset = ['iata_airline'], keep = 'first', inplace = True) 

#df_airlines_three

df_airlines_merge = pd.merge(df_airlines_three, df_country, on='country_name', how='inner') 
#df_airlines_merge

In [22]:
#POPULATE AIRLINE TABLE
#TABLE 5

df_airline = df_airlines_merge[['iata_airline','icao_airline','airline_name','active','country_code']]
df_airline = df_airline.drop_duplicates(keep='first')
# df_airline


#df_airline=df_airline.drop_duplicates(subset = ['iata_airline'], keep = 'first', inplace = True) 
df_airline_final = df_airline


#WRITING THE TABLE TO DATABASE
df_airline_final.to_sql(con=dbConnection,name='airline',if_exists='append',index=False)

display(df_airline_final)
print("airline table added to database successfully")


,iata_airline,icao_airline,airline_name,active,country_code
0,1T,RNX,1Time Airline,Y,ZA
1,MN,CAW,Comair,Y,ZA
2,ID,ITK,Interlink Airlines,Y,ZA
3,JE,MNO,Mango,Y,ZA
4,CE,NTW,Nationwide Airlines,Y,ZA
...,...,...,...,...,...
667,IY,IYE,Yemenia,Y,YE
668,FU,FXX,Felix Airways,Y,YE
669,ML,MAV,Maldivo Airlines,Y,MV
670,DN,SGG,Senegal Airlines,Y,SN


airline table added to database successfully


In [25]:
#READING THE SKYTRAX SCRAPPED CSV FILES FOR ALL 10 AIRLINE REVIEWS

#Reading the Airline CSV files
def readData():
    colnames = ['airline_name','overall_score','review_title','reviewer_country','review_date','aircraft_model','travel_type','seat_type','route','date_flown','seat_comfort_rating','service_rating','food_rating','entertainment_rating','groundservice_rating','wifi_rating','value_rating','recommended']
    Afrance = pd.read_csv(f"{url}/Web_Scrapping_Data/Airlines_Data/air-france.csv",header = None,names = colnames)
    Ana = pd.read_csv(f"{url}/Web_Scrapping_Data/Airlines_Data/ana-all-nippon-airways.csv",header = None,names = colnames)
    Emirates = pd.read_csv(f"{url}/Web_Scrapping_Data/Airlines_Data/emirates.csv",header = None,names = colnames)
    Japan = pd.read_csv(f"{url}/Web_Scrapping_Data/Airlines_Data/japan-airlines.csv",header = None,names = colnames)
    Korean = pd.read_csv(f"{url}/Web_Scrapping_Data/Airlines_Data/korean-air.csv",header = None,names = colnames)
    Qantas = pd.read_csv(f"{url}/Web_Scrapping_Data/Airlines_Data/qantas-airways.csv",header = None,names = colnames)
    Qatar = pd.read_csv(f"{url}/Web_Scrapping_Data/Airlines_Data/qatar-airways.csv",header = None,names = colnames)
    Singapore = pd.read_csv(f"{url}/Web_Scrapping_Data/Airlines_Data/singapore-airlines.csv",header = None,names = colnames)
    Swiss = pd.read_csv(f"{url}/Web_Scrapping_Data/Airlines_Data/swiss-international-air-lines.csv",header = None,names = colnames)
    Turkish = pd.read_csv(f"{url}/Web_Scrapping_Data/Airlines_Data/turkish-airlines.csv",header = None,names = colnames)
    df = pd.concat([Afrance,Afrance,Ana,Emirates,Japan,Korean,Qantas,Qatar,Singapore,Swiss,Turkish])
    return df

In [26]:
#CLEANING THE DATE

#READ DATA
df_airline_reviews = readData()
df_airline_reviews

#DROP DUPLICATES
df_airline_reviews = df_airline_reviews.drop_duplicates(keep='first')

#DROP AIRLINE_REVIEWS ROW 0
df_airline_reviews = df_airline_reviews.drop([0])


#STRIP CHARACTERS
df_airline_reviews.review_title = df_airline_reviews.review_title.apply(lambda x: stripChar(x))
df_airline_reviews.reviewer_country = df_airline_reviews.reviewer_country.apply(lambda x: stripChar(x))
#display(df_airline_reviews)


#FILL EMPTY CELLS WITH VAL 0
df_airline_reviews.recommended = df_airline_reviews.recommended.fillna(value = 0)
#display(df_airline_reviews)

#FILL 1 TO CELLS WITH VAL YES
df_airline_reviews.recommended = df_airline_reviews.recommended.eq('yes').mul(1)
#display(df_airline_reviews)

#DATETIME CHANGES
df_airline_reviews['review_date'] = pd.to_datetime(df_airline_reviews['review_date'])
# df_airline_reviews['year'] = df_airline_reviews['review_date'].dt.year
# df_airline_reviews['month'] = df_airline_reviews['review_date'].dt.month
# df_airline_reviews.drop(['date_flown'], axis=1)


display(df_airline_reviews)


,airline_name,overall_score,review_title,reviewer_country,review_date,aircraft_model,travel_type,seat_type,route,date_flown,seat_comfort_rating,service_rating,food_rating,entertainment_rating,groundservice_rating,wifi_rating,value_rating,recommended
1,air france,10.0,Plane was spotless,New Caledonia,2022-05-05,NaN,Solo Leisure,Economy Class,Paris to Lyon,May 2022,5,5,5,0,5,0,5,1
2,air france,1.0,got nothing but the run around,New Caledonia,2022-04-19,NaN,Solo Leisure,Economy Class,NaN,September 2021,0,0,0,0,0,0,1,0
3,air france,2.0,would not recommend,New Caledonia,2022-04-15,NaN,Family Leisure,Economy Class,Tel Aviv to Toronto via Paris,April 2022,3,1,2,3,1,4,1,0
4,air france,1.0,didn’t bother to put my bag on the aircraft,New Caledonia,2022-04-06,A319,Business,Economy Class,Paris to Newcastle,April 2022,1,1,1,0,1,1,1,0
5,air france,7.0,nothing by way of compensation,New Caledonia,2022-04-05,Boeing 777-200 / 777-300,Business,Business Class,Conakry to Washington via Paris,December 2021,4,4,4,4,2,4,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,turkish airlines,1.0,most uncomfortable seats,United States,2018-07-16,NaN,Couple Leisure,Business Class,Mumbai to Helsinki via Instanbul,July 2018,1,2,2,2,1,0,1,0
996,turkish airlines,3.0,never walking the cabin,United States,2018-07-16,Boeing 777,Solo Leisure,Business Class,Manila to Istanbul,July 2018,3,1,1,2,2,1,1,0
997,turkish airlines,1.0,at the back of the plane chatting,United States,2018-07-15,Boeing 777-300ER,Solo Leisure,Economy Class,Tel Aviv to Manila via Istanbul,June 2018,3,1,2,4,3,0,1,0
998,turkish airlines,1.0,"Well, money, obviously",United States,2018-07-15,NaN,Couple Leisure,Premium Economy,Kayseri to Bucharest via Istanbul,July 2018,1,1,1,1,1,1,1,0


In [30]:
#INGESTING ONLY THOSE RECORDS WHOSE PARENT IS PRESENT IN THE AIRLINES TABLE

df_new = df_airline_reviews[['airline_name']]
df_new['value']=df_airline_reviews.airline_name.str.lower().isin(list(df_airline_final.airline_name.str.lower()))
df_new=df_new.loc[df_new['value'].isin([True])]
df_new=df_new.drop_duplicates(keep='first')
#df_new

df_airline_final['airline_name'] = df_airline_final['airline_name'].str.lower()
df_airline_final

df_airlines_iata = df_airline_final[['airline_name','iata_airline']]
df_airlines_iata

#MERGING 
merge_df = pd.merge(df_airline_reviews, df_airlines_iata, on='airline_name', how='left')
df_review_main = merge_df
#df_review_main

/var/folders/p9/xt4cp34n3fxdkk5sts27rrzw0000gp/T/ipykernel_18482/840913480.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [31]:
#POPULATE AIRLINE_REVIEWS TABLE
#TABLE 6

df_review_final = df_review_main[['iata_airline','review_title','review_date','seat_comfort_rating','service_rating','food_rating','entertainment_rating','groundservice_rating','wifi_rating','value_rating','overall_score','recommended']]
df_review_final = df_review_final.drop_duplicates(keep='first')
display(df_review_final)


#WRITING THE TABLE TO DATABASE
df_review_final.to_sql(con=dbConnection,name='airline_review',if_exists='append',index=False)

print("airline_review table added to database successfully")

,iata_airline,review_title,review_date,seat_comfort_rating,service_rating,food_rating,entertainment_rating,groundservice_rating,wifi_rating,value_rating,overall_score,recommended
0,AF,Plane was spotless,2022-05-05,5,5,5,0,5,0,5,10.0,1
1,AF,got nothing but the run around,2022-04-19,0,0,0,0,0,0,1,1.0,0
2,AF,would not recommend,2022-04-15,3,1,2,3,1,4,1,2.0,0
3,AF,didn’t bother to put my bag on the aircraft,2022-04-06,1,1,1,0,1,1,1,1.0,0
4,AF,nothing by way of compensation,2022-04-05,4,4,4,4,2,4,3,7.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
8304,TK,most uncomfortable seats,2018-07-16,1,2,2,2,1,0,1,1.0,0
8305,TK,never walking the cabin,2018-07-16,3,1,1,2,2,1,1,3.0,0
8306,TK,at the back of the plane chatting,2018-07-15,3,1,2,4,3,0,1,1.0,0
8307,TK,"Well, money, obviously",2018-07-15,1,1,1,1,1,1,1,1.0,0


airline_review table added to database successfully


In [33]:
#POPULATE TABLE 7

df_aircraft = pd.read_csv(f"{url}/aircraft/aircraft_types_02.txt",header = 0)
df_aircraft = df_aircraft.drop(['emty','Unnamed: 6'],axis=1)
df_aircraft = df_aircraft.dropna(how='any')
df_aircraft = df_aircraft[df_aircraft.icao_aircraft != 'Nan']
df_aircraft = df_aircraft[df_aircraft.name != 'Nan']
df_aircraft = df_aircraft[df_aircraft.iata_aircraft != 'Nan']
df_aircraft = df_aircraft[df_aircraft.country != 'Nan']
df_aircraft = df_aircraft[df_aircraft.capacity != 'Nan']
df_aircraft = df_aircraft.drop(['country'],axis=1)
len(list(df_aircraft['iata_aircraft'].unique()))
df_aircraft.rename(columns={'name':'aircraft_name'},inplace=True )
#df_aircraft.country = df_aircraft.country.apply(lambda x: stripChar(x))
df_aircraft = df_aircraft[['iata_aircraft','icao_aircraft','aircraft_name','capacity']]
display(df_aircraft)

#WRITING THE TABLE TO DATABASE
df_aircraft.to_sql(con=dbConnection,name='aircraft',if_exists='append',index=False)


print("aircraft table added to database successfully")

,iata_aircraft,icao_aircraft,aircraft_name,capacity
0,AT5,AT45,Aerospatiale/Alenia ATR 42-500,50
2,AT7,AT72,Aerospatiale/Alenia ATR 72,74
3,SSC,CONC,Aerospatiale/BAC Concorde,128
4,AB3,A30B,Airbus A300 pax,200
5,AB6,A306,Airbus A300-600 pax,266
...,...,...,...,...
212,S61,S61,Sikorsky S-61,30
213,S76,S76,Sikorsky S-76,13
214,T20,T204,Tupolev Tu-204 / Tu-214,212
215,TU3,T134,Tupolev Tu134,96


IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry 'AB3' for key 'aircraft.PRIMARY'")
[SQL: INSERT INTO aircraft (iata_aircraft, icao_aircraft, aircraft_name, capacity) VALUES (%(iata_aircraft)s, %(icao_aircraft)s, %(aircraft_name)s, %(capacity)s)]
[parameters: ({'iata_aircraft': 'AT5', 'icao_aircraft': 'AT45', 'aircraft_name': ' Aerospatiale/Alenia ATR 42-500', 'capacity': '50'}, {'iata_aircraft': 'AT7', 'icao_aircraft': 'AT72', 'aircraft_name': ' Aerospatiale/Alenia ATR 72', 'capacity': '74'}, {'iata_aircraft': 'SSC', 'icao_aircraft': 'CONC', 'aircraft_name': ' Aerospatiale/BAC Concorde', 'capacity': '128'}, {'iata_aircraft': 'AB3', 'icao_aircraft': 'A30B', 'aircraft_name': ' Airbus A300 pax', 'capacity': '200'}, {'iata_aircraft': 'AB6', 'icao_aircraft': 'A306', 'aircraft_name': ' Airbus A300-600 pax', 'capacity': '266'}, {'iata_aircraft': '310', 'icao_aircraft': 'A310', 'aircraft_name': ' Airbus A310 all pax models', 'capacity': '198'}, {'iata_aircraft': '312', 'ic

In [34]:
#ADDING THE TWITTER DATA

#FUNCTION TOFIND AIRLINE NAMES FROM TWITTER TEXT

def airlineName(tweet):
        text = (tweet.content).lower()
        print(f"\ntweet id --> {text}")
        print(type(text))
        iata_airline = [];
        
        
        
        if(("@qatar" in text) and ("QR") not in iata_airline):
            iata_airline.append("QR")
        elif(("@british" in text) and ("BA") not in iata_airline):
            iata_airline.append("BA")
        elif(("@lufthanza" in text) and ("LH") not in iata_airline):
            iata_airline.append("LH")  
        elif(("@emirates" in text) and ("EK") not in iata_airline):
            iata_airline.append("EK")  
        elif(("@singapore" in text) and ("SQ") not in iata_airline):
            iata_airline.append("SQ")  
        elif(("@turkish_airways" in text) and ("TK") not in iata_airline):
            iata_airline.append("TK")
        elif(("@american_air" in text) and ("AA") not in iata_airline):
            iata_airline.append("AA")
        elif(("@flyana_official" in text) and ("NH") not in iata_airline):
            iata_airline.append("NH")
        elif(("@qantas" in text) and ("QF") not in iata_airline):
            iata_airline.append("QF")
        elif(("@flyswiss" in text) and ("LS") not in iata_airline):
            iata_airline.append("LS")

        
            
        airline_str = ''.join(str(x) for x in iata_airline)
        return airline_str

In [28]:
#IITERATE THROUGH TWEETS USING SNSCRAPE

keywords =  "(airlines or airline or flight or airways)"
#keywords =  "((qatar_airways or qatar or british or british_air or singapore or turkish_airways or american_air or flyana_official or qantas or flyswiss) and (airlines or airline or flight or airways))"
tweet_user = []
tweets = []
tweet_tags = []
tweet_mentions = []
airlines = []

tweet_airline = []

for i, tweet in enumerate(sntwitter.TwitterSearchScraper(keywords).get_items()):

    if i >= 1000:
        break
    else:
        iata_airline = airlineName(tweet)
        tweet_id_str = (str)(tweet.id);
        print(type(tweet.date))
        print(f"tweet id --> {tweet.user.id}")
        
    
        tweets.append([tweet.id,tweet.user.username,tweet.content,iata_airline,tweet.likeCount,tweet.date,tweet.user.displayname,tweet.user.id,tweet.user.profileImageUrl,tweet.user.description, tweet.user.followersCount, tweet.user.friendsCount,tweet.user.verified])

        

NameError: name 'sntwitter' is not defined

In [35]:
#TABLE TWEETS MAIN TABLE

cols_tweet = ["tweet_id","twitter_handle","tweet_text","iata_airline","like_count","created_at","user_display_name","user_id","user_profile_image_url","user_description","user_follower_count","user_following_count","verified_account"]
#tweets.append([tweet.id,tweet.user.username,tweet.content,airline_review,tweet.likeCount,tweet.date])
        
df_tweets = pd.DataFrame(tweets, columns = cols_tweet)

#REMOVE ALL THE REVIEWS EXCLUDING THE RELEVANT AIRLINES
df_tweets = df_tweets.loc[df_tweets['iata_airline']!=""]


print("MAIN TWEET TABLE IN THE DATABASE")
display(df_tweets)



NameError: name 'tweets' is not defined

In [36]:
#POPULATE TABLE TWEET_USER
#TABLE 9

#cols_tweet_user = ["twitter_handle","user_dispay_name","user_id","user_profile_image_url","user_description","user_followers_count","user_following_count"]
df_tweets_user = df_tweets[['twitter_handle','user_display_name','user_id','user_profile_image_url','user_description','user_follower_count','user_following_count','verified_account']]

#DROPPING DUPLICATE TWITTER_HANDLE INFO
df_tweets_user=df_tweets_user.drop_duplicates('twitter_handle',keep='first')

display(df_tweets_user)
#APPENDING DATA TO MYSQL DATABASE
df_tweets_user.to_sql(con=dbConnection,name='tweet_user',if_exists='append',index=False)


print("tweet_user data added successfully")
print(len(df_tweets_user))

NameError: name 'df_tweets' is not defined

In [37]:
#POPULATE TABLE TWEET
#TABLE 10

df_tweets_final = df_tweets[["tweet_id","twitter_handle","tweet_text","iata_airline","like_count","created_at"]]
display(df_tweets_final)


#APPENDING DATA TO MYSQL DATABASE
df_tweets_final.to_sql(con=dbConnection,name='tweet',if_exists='append',index=False)

print("tweet data added successfully")
print(len(df_tweets_final))

NameError: name 'df_tweets' is not defined